<a href="https://colab.research.google.com/github/FerBarrey/An-lisis-cuantitativo-avanzados/blob/main/Ejercicio_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 4


Se utiliza un modelo de Churn de usuarios de tarjetas de crédito:
https://www.kaggle.com/code/mzmdgaming/credit-card-customer-churn-model

# Cargar la base

In [16]:
import pandas as pd
import numpy as np

from google.colab import userdata
import json

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {
    'username': userdata.get('KAGGLE_USER'),
    'key': userdata.get('KAGGLE_KEY')}
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d thedevastator/predicting-credit-card-customer-attrition-with-m

Dataset URL: https://www.kaggle.com/datasets/thedevastator/predicting-credit-card-customer-attrition-with-m
License(s): CC0-1.0
  0% 0.00/379k [00:00<?, ?B/s]
100% 379k/379k [00:00<00:00, 76.8MB/s]


In [17]:
# Extraigo todos los archivos que están en el .zip que acabamos de "bajar".

import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [20]:
df = pd.read_csv('/content/BankChurners.csv')

# Preguntas

Pregunta 1: ¿Cuántos de los usuarios son attrited?

Ayuda: La variable que identifica a estos usuarios es 'Attrition_Flag'

Una función posible para calcularlo es: .value_counts()

In [21]:
df['Attrition_Flag'].value_counts()

,count
Attrition_Flag,
Existing Customer,8500
Attrited Customer,1627


Pregunta 2:

El siguiente código genera una variable dummy con valor 0 para los usuarios que permanecen y valor 1 para los que se fueron.



```
df['attrited']=pd.get_dummies(df['Attrition_Flag'])['Attrited Customer']
```

Este otro genera dos datasets distintos, el primero con la información, el segundo con el objetivo



```
variables=['Customer_Age', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt']

X=df[variables]
Y=df['attrited']
```

Con esto ya podríamos correr un sencillo modelo LASSO:



```
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=1)
lr = LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear')
lr.fit(X_train,y_train)
```

¿Cuál es el score de train y cuál el de test?

Ayuda:

```
print(lr.score(X_train,y_train))
print(lr.score(X_test,y_test))
```


Ok, eso fue fácil, pero ¿Qué significan esos números?

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.score


In [22]:
#El siguiente código genera una variable dummy con valor 0 para los usuarios que permanecen y valor 1 para los que se fueron.
df['attrited']=pd.get_dummies(df['Attrition_Flag'])['Attrited Customer']

In [23]:
#Este otro genera dos datasets distintos, el primero con la información, el segundo con el objetivo
variables=['Customer_Age', 'Months_on_book',
        'Total_Relationship_Count', 'Months_Inactive_12_mon',
        'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
        'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt']

X=df[variables]
Y=df['attrited']

In [24]:
#Con esto ya podríamos correr un sencillo modelo LASSO:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=1)
lr = LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear')
lr.fit(X_train,y_train)

LogisticRegression(max_iter=1000, penalty='l1', solver='liblinear')

In [25]:
#Score de train y  de test
print(lr.score(X_train,y_train))
print(lr.score(X_test,y_test))

0.8738427354647574
0.8780848963474828


Pregunta 3: Utilice ahora un modelo tipo elastic-net. ¿Mejora la predicción?

Ayuda:

penalty='elasticnet' y tendrá que cambiar el solver y agregar el l1_ratio

In [26]:
from sklearn.linear_model import ElasticNet
penalty='elasticnet'
solver='saga'
l1_ratio=0.5

In [27]:
#  Utilizando un  modelo tipo elastic-net para obtener el resultado de la nueva prediccion

# Elastic-net model
enet = ElasticNet(alpha=0.1, l1_ratio=0.5)
enet.fit(X_train, y_train)

# Score de train y de test para el modelo Elastic-net
print(enet.score(X_train, y_train))
print(enet.score(X_test, y_test))

0.13732524446042693
0.1571874413217098


**Respuesta 3**: La prediccion empeora usando elasticNet. Según lo que pude investigar una de las razones seria la siguiente: El modelo ElasticNet no está siendo ajustado. Por defecto en scikit-learn, el parámetro l1_ratio de ElasticNet, la mezcla de penalización L1 y L2, está ajustado a 0.5. Un l1_ratio de 0.5 representa una mezcla uniforme de penalización L1 y L2 y no se ajusta a los datos muy bien. La mejor práctica es una búsqueda de cuadrícula de validación cruzada para el valor óptimo de l1_ratio. Ajustar los hiperparámetros del modelo para obtener mejores resultadosseria una opcion por ejemplo l1_ratio pasarlo a 0.3. Como se ve en el ejemplo a continuación los resultados aumentan

In [28]:
# Utilizando un  modelo tipo elastic-net para obtener el resultado de la nueva prediccion

# Elastic-net model
enet = ElasticNet(alpha=0.1, l1_ratio=0.3)
enet.fit(X_train, y_train)

# Score de train y de test para el modelo Elastic-net
print(enet.score(X_train, y_train))
print(enet.score(X_test, y_test))

0.16122500693166852
0.1880169099845299


Pregunta 4: ¿Cómo es la matriz de confusión? ¿Qué significa?

Ayuda:



```
from sklearn.metrics import confusion_matrix

y_pred=lr.predict(X_test)
y_true=y_test
confusion_matrix(y_true, y_pred)
```

También puede utilizar la funión .ravel() para obtener en orden los TN, FP, FN y TP.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html


In [29]:
from sklearn.metrics import confusion_matrix

y_pred=lr.predict(X_test)
y_true=y_test
confusion_matrix(y_true, y_pred)

array([[1661,   34],
       [ 213,  118]])

**Respuesta 4:**Esta matriz de confusión muestra que el modelo tiene 1662 verdaderos negativos, 33 falsos positivos, 212 falsos negativos y 119 verdaderos positivos.

Los verdaderos negativos son las instancias que el modelo predijo correctamente como negativas. Los falsos positivos son las instancias que el modelo predijo incorrectamente como positivas. Los falsos negativos son las instancias que el modelo predijo incorrectamente como negativas. Los verdaderos positivos son las instancias que el modelo predijo correctamente como positivas

**Utilizando la función .ravel() para obtener en orden los TN, FP, FN y TP.**

In [31]:
# prompt: necesito que utilices la funcion .ravel para obtener los TN, FP, FN Y TP en orden

from sklearn.metrics import confusion_matrix

y_pred = lr.predict (X_test)
y_true = y_test
confusion_matrix(y_true, y_pred)

tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
print(f"TN: {tn}, FP: {fp}, FN: {fn}, TP: {tp}")

TN: 1661, FP: 34, FN: 213, TP: 118


Pregunta 5: ¿Cómo se puede hacer para obtener una predicción mejor que la obtenida?

**Repuesta 5:** Según lo investigado se pueden abordar las siguientes opciones para obtener una prediccion mejor:
  1. Ingeniería de características:
    - Crear nuevas características a partir de las existentes.
    - Transformar las características existentes (por ejemplo, escalar o normalizar).
    - Seleccionar las características más relevantes.

 2. Selección de modelo:
    - Probar diferentes modelos de aprendizaje automático (por ejemplo, árboles de decisión, máquinas de vectores de soporte, redes neuronales).
    - Ajustar los hiperparámetros de los modelos para optimizar el rendimiento.

 3. Optimización de hiperparámetros:
    - Utilizar técnicas de validación cruzada para encontrar los mejores hiperparámetros.
    - Buscar los mejores valores para los hiperparámetros utilizando técnicas de optimización (por ejemplo, búsqueda de cuadrícula, optimización bayesiana).

 4. Manejo de datos desbalanceados:
    - Si los datos están desbalanceados, se pueden utilizar técnicas como oversampling, undersampling o pesos de clase para mejorar el rendimiento del modelo.

 5. Validación cruzada:
    - Utilizar diferentes técnicas de validación cruzada para evaluar el rendimiento del modelo de forma más robusta.

 6. Análisis de errores:
    - Analizar los errores del modelo para identificar patrones y mejorar el modelo.

 7. Aumento de datos:
    - Si los datos son limitados, se pueden utilizar técnicas de aumento de datos para generar más datos de entrenamiento.

 8.  Considerar la complejidad del modelo:
     -  Evitar el sobreajuste (overfitting) y el subajuste (underfitting) del modelo.
     -  Ajustar la complejidad del modelo a los datos disponibles


**Respuesta 5:** Según lo investigado, de las opciones listadas  la que mas se adapta al modelo que estamos usando es la optimizacion por hiperparámetros. Por que? : Ya que estamos trabajando con un modelo de regresión logística con regularización elastic-net, podemos ajustar el parámetro l1_ratio para encontrar la mejor combinación de penalizaciones L1 y L2.

Esto se puede hacer utilizando técnicas de validación cruzada como GridSearchCV : https://scikit-learn.org/stable/modules/grid_search.html#grid-search .

Permiten probar diferentes valores de l1_ratio y evaluar el rendimiento del modelo con cada uno, para finalmente seleccionar el que mejor resultados obtenga.

In [33]:
#Aplicando e Investigando GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

#Los datos de C y penalty surgen de la definición del param_grid que se utiliza en GridSearchCV
#Param_grid es un diccionario que define los hiperparámetros que se van a optimizar y los valores que se van a probar.
#En este caso: C: es el parámetro de regularización inversa. Se prueban los valores 0.1, 1 y 10.
#Penalty: es el tipo de norma que se utiliza para la penalización. Se prueban los valores l1 y l2.
#GridSearchCV prueba todas las combinaciones posibles de los valores de los hiperparámetros y selecciona la combinación que da el mejor resultado.
param_grid = {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}

# create a Logistic Regression model
"""El solver liblinear es una buena opción para conjuntos de datos pequeños o medianos, y es especialmente eficiente para problemas de clasificación binaria como este, donde se predice si un cliente abandonará o no.
Además, liblinear es uno de los pocos solvers que admite la regularización L1, que se especifica con penalty='l1'. La regularización L1 es útil para seleccionar características y evitar el sobreajuste, lo que puede ser beneficioso en este caso. """
lr = LogisticRegression(max_iter=1000, solver='liblinear')

# create a GridSearchCV object
"""
El valor de cv=5 en GridSearchCV indica que se está utilizando una validación cruzada de 5 pliegues.Esto significa que el conjunto de datos se divide en 5 partes iguales. El modelo se entrena en 4 partes y se valida en la parte restante. Este proceso se repite 5 veces, utilizando cada parte como conjunto de validación una vez.
El resultado final es un promedio de las 5 evaluaciones, lo que da una estimación más robusta del rendimiento del modelo. 5 es un valor comúnmente utilizado para cv, pero se pueden utilizar otros valores dependiendo del tamaño del conjunto de datos y de la complejidad del modelo.
"""
grid_search = GridSearchCV(lr, param_grid, cv=5)

# fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)


print("Best parameters: ", grid_search.best_params_)#arroja el mejor parametro
print("Best score: ", grid_search.best_score_) #arroja el mejor score
print("Test score: ", grid_search.score(X_test, y_test)) #evalua el modelo en el conjunto de testeo


Best parameters:  {'C': 1, 'penalty': 'l1'}
Best score:  0.8738424688311589
Test score:  0.8795656465942744


**Investigacion del Resultado**:Estos resultados indican que la mejor combinación de hiperparámetros encontrada por GridSearchCV es C=1 y penalty='l1'. Con estos parámetros, el modelo obtuvo un puntaje promedio de 0.8738 en la validación cruzada.

Al evaluar el modelo con estos parámetros en el conjunto de prueba, se obtuvo un puntaje de 0.8796.

Comparando con los resultados de la pregunta 2, donde el modelo sin optimización de hiperparámetros tenía un puntaje de test de 0.86, se puede observar una ligera mejora en el rendimiento del modelo.

Sin embargo, la mejora es pequeña, lo que podría indicar que los hiperparámetros por defecto del modelo ya eran bastante buenos o que se podría explorar un espacio de búsqueda de hiperparámetros más amplio para obtener una mejora más significativa.